# AOTU input and output ROIs

In [16]:
# Import standard libraries
import os, datetime
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle as pl
import igraph
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import bokeh
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', 'matplotlib')
%matplotlib inline 

import bokeh.palettes
from bokeh.plotting import figure, show, output_notebook
output_notebook()
import seaborn as sb
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz, load_npz
# import cairo
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import zscore
from scipy.sparse.linalg import norm
import networkx as nx
from pyvis.network import Network

## Load environment variables from .env file and find project root
import sys
from dotenv import load_dotenv, find_dotenv
load_dotenv()
PROJECT_ROOT = Path(find_dotenv()).parent
# data_path = Path(PROJECT_ROOT, 'results', 'eyemap')
store_path = Path(PROJECT_ROOT, 'results','AOTU_Connectivity')
sys.path.append(str(PROJECT_ROOT.joinpath('src')))


from utils import olc_client
c = olc_client.connect(verbose=True)

from utils.celltype_conn_by_roi import CelltypeConnByRoi
from utils.celltype_conn_plotter import CelltypeConnPlotter
from utils.plotter import plot_cns, save_figure, get_skeletons, get_skeleton, get_meshes, get_mesh, show_figure
from utils.helper import slugify
from utils.neuron_bag import NeuronBag

# Import neuPrint specific libraries
from neuprint import Client, fetch_neurons, NeuronCriteria as NC, fetch_neurons, fetch_simple_connections, fetch_adjacencies, connection_table_to_matrix, merge_neuron_properties, NotNull, fetch_synapse_connections, fetch_neurons,fetch_primary_rois, fetch_all_rois, fetch_synapses, fetch_roi_hierarchy
from neuprint.utils import connection_table_to_matrix

Loading BokehJS ...

Connected to https://neuprint-cns.janelia.org[cns].
Client: neuprint-python v1.7.4
User: aishahamid201@gmail.com [readwrite]



In [2]:
PROJECT_ROOT

PosixPath('/Users/hamida/Documents/GitHub/male-drosophila-visual-system-connectome-code')

In [40]:
# Specify Neurons of Interest by instance or type (save neuron properties & synapse ROIs if you wish):

## For only AOTU neurons on right side:
AOTU_criteria = (NC(type='^AOTU.*'));
AOTU_neuron_df,roi_counts_df = fetch_neurons(AOTU_criteria)

In [41]:
## To save neuron properties and synapse ROIs as .csv files:
AOTU_neuron_df.to_csv(PROJECT_ROOT/'AOTU_results/LC10a_R_NeuronProperties.csv')#
roi_counts_df.to_csv(PROJECT_ROOT/'AOTU_results/LC10a_R_SynapsePerROICounts.csv')

## to list LC10s, uncomment:
AOTU_neuron_df

,bodyId,instance,type,pre,post,downstream,upstream,size,status,statusLabel,...,dimorphism,assignedOlHex1,trumanHl,entryNerve,predictedNtConfidence,somaNeuromere,somaSide,hemibrainBodyId,inputRois,outputRois
0,10005,AOTU019_R,AOTU019,2837,31981,23423,31981,11847745920,Traced,Roughly traced,...,None,None,None,None,0.791933,None,R,None,"[AOTU(R), CRE(R), CentralBrain, CentralBrain-u...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
1,10031,AOTU041_R,AOTU041,2134,20586,13446,20586,5326291658,Traced,Roughly traced,...,None,None,None,None,0.829691,None,R,None,"[AOTU(L), AOTU(R), CRE(L), CRE(R), CentralBrai...","[AOTU(L), AOTU(R), CentralBrain, CentralBrain-..."
2,10070,AOTU019_L,AOTU019,2836,30714,23596,30714,11809707230,Traced,Roughly traced,...,None,None,None,None,0.767168,None,L,None,"[AOTU(L), CRE(L), CentralBrain, CentralBrain-u...","[AOTU(L), CRE(L), CentralBrain, CentralBrain-u..."
3,10148,AOTU041_L,AOTU041,2336,19960,14187,19960,5592462575,Traced,Roughly traced,...,None,None,None,None,0.778527,None,L,None,"[AOTU(L), AOTU(R), BU(L), CRE(L), CentralBrain...","[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr..."
4,10212,AOTU042_L,AOTU042,3579,10679,22690,10679,4312493196,Traced,Roughly traced,...,None,None,None,None,0.824498,None,L,None,"[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr...","[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,555803,AOTU024_R,AOTU024,814,2240,6989,2240,964400165,Traced,Roughly traced,...,None,None,None,None,0.480699,None,R,None,"[AOTU(R), ATL(L), ATL(R), CRE(R), CentralBrain...","[AOTU(R), ATL(L), ATL(R), CentralBrain, Centra..."
308,557095,AOTU008_L,AOTU008,392,2167,3198,2167,832073642,Traced,Roughly traced,...,sexually dimorphic,None,None,None,0.545708,None,L,None,"[AOTU(L), AOTU(R), CentralBrain, CentralBrain-...","[AOTU(L), CentralBrain, CentralBrain-unspecifi..."
309,565505,AOTU008_R,AOTU008,315,2727,2802,2727,502528277,Traced,Roughly traced,...,sexually dimorphic,None,None,None,0.619007,None,R,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
310,902249,AOTU038_R,AOTU038,111,459,704,459,278656951,Traced,Roughly traced,...,None,None,None,None,0.416629,None,R,None,"[AOTU(R), AVLP(R), CentralBrain, CentralBrain-...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."


In [28]:
print(AOTU_neuron_df.columns.tolist())

['bodyId', 'instance', 'type', 'pre', 'post', 'downstream', 'upstream', 'size', 'status', 'statusLabel', 'cellBodyFiber', 'somaLocation', 'roiInfo', 'tosomaLocation', 'predictedNt', 'totalNtPredictions', 'mancSerial', 'mcnsSerial', 'locationType', 'group', 'location', 'birthtime', 'receptorType', 'celltypeTotalNtPredictions', 'posteriorCervicalRootPosition', 'synweight', 'mancGroup', 'consensusNt', 'celltypePredictedNtConfidence', 'mancBodyid', 'serialMotif', 'flywireId', 'anteriorCervicalRootPosition', 'assignedOlHex2', 'class', 'synonyms', 'exitNerve', 'flywireType', 'trumanHlVncOnly', 'matchingNotes', 'subclass', 'itoleeHl', 'supertype', 'fruDsx', 'celltypePredictedNt', 'hemibrainType', 'mancType', 'rootSide', 'superclass', 'dimorphism', 'assignedOlHex1', 'trumanHl', 'entryNerve', 'predictedNtConfidence', 'somaNeuromere', 'somaSide', 'hemibrainBodyId', 'inputRois', 'outputRois']


In [42]:
#print(sorted(AOTU_neuron_df['type'].unique()))
print(sorted(AOTU_neuron_df['inputRois']))


[['AL(L)', 'AOTU(R)', 'AVLP(L)', 'AVLP(R)', 'CRE(R)', 'CentralBrain', 'CentralBrain-unspecified', 'EPA(L)', 'EPA(R)', 'FLA(L)', 'GOR(L)', 'GOR(R)', 'IB', 'ICL(L)', 'ICL(R)', 'LAL(L)', 'LAL(R)', 'PED(R)', 'PLP(R)', 'PVLP(L)', 'PVLP(R)', 'SCL(R)', 'SIP(R)', 'SMP(R)', 'SPS(R)', 'VES(L)', 'VES(R)', 'WED(L)', "a'L(R)", 'aL(R)', "b'L(R)"], ['AMMC(L)', 'AOTU(L)', 'AVLP(L)', 'CentralBrain', 'CentralBrain-unspecified', 'LAL(L)', 'PLP(L)', 'SAD', 'SIP(L)', 'WED(L)'], ['AMMC(L)', 'AOTU(L)', 'CentralBrain', 'CentralBrain-unspecified', 'GNG', 'IB', 'IPS(L)', 'PLP(L)', 'SAD', 'SIP(L)', 'SPS(L)', 'SPS(R)', 'WED(L)', 'aL(L)'], ['AMMC(L)', 'AOTU(L)', 'CentralBrain', 'CentralBrain-unspecified', 'GNG', 'IPS(L)', 'IPS(R)', 'PLP(L)', 'SAD', 'SIP(L)', 'SPS(L)', 'SPS(R)', 'WED(L)'], ['AMMC(L)', 'AOTU(L)', 'CentralBrain', 'CentralBrain-unspecified', 'GNG', 'LH(L)', 'PLP(L)', 'SAD', 'SIP(L)', 'WED(L)', "a'L(L)"], ['AMMC(L)', 'AOTU(L)', 'CentralBrain', 'CentralBrain-unspecified', 'GNG', 'PLP(L)', 'SAD', 'SIP(L)

In [38]:
unique_celltypes = AOTU_neuron_df['type'].unique()
unique_celltypes

array(['AOTU019', 'AOTU041', 'AOTU042', 'AOTU023', 'AOTU012', 'AOTU035',
       'AOTU005', 'AOTU100m', 'AOTU016_c', 'AOTU061', 'AOTU052',
       'AOTU103m', 'AOTU063_b', 'AOTU064', 'AOTU033', 'AOTU101m',
       'AOTU063_a', 'AOTU049', 'AOTU027', 'AOTU014', 'AOTU015', 'AOTU046',
       'AOTU050', 'AOTU024', 'AOTU059', 'AOTU009', 'AOTU008', 'AOTU045',
       'AOTU065', 'AOTU007_c', 'AOTU002_c', 'AOTU029', 'AOTU016_a',
       'AOTU002_a', 'AOTU026', 'AOTU020', 'AOTU017', 'AOTU028',
       'AOTU016_b', 'AOTU036', 'AOTU006', 'AOTU043', 'AOTU048', 'AOTU013',
       'AOTU051', 'AOTU034', 'AOTU011', 'AOTU002_b', 'AOTU022', 'AOTU062',
       'AOTU032', 'AOTU001', 'AOTU007_b', 'AOTU047', 'AOTU003',
       'AOTU007_a', 'AOTU054', 'AOTU053', 'AOTU030', 'AOTU056', 'AOTU055',
       'AOTU060', 'AOTU102m', 'AOTU038', 'AOTU058', 'AOTU037', 'AOTU018',
       'AOTU004', 'AOTU039', 'AOTU040', 'AOTU021', 'AOTU025'],
      dtype=object)

In [43]:

# Step 1: Group input ROIs by unique AOTU types
type_to_rois = (
    AOTU_neuron_df
    .groupby('type')['inputRois']
    .apply(lambda x: set().union(*x.dropna()))
    .explode()
    .reset_index()
)

# Step 2: Build binary matrix (type × ROI)
binary_matrix = pd.crosstab(type_to_rois['inputRois'], type_to_rois['type'])

# Step 3: Sort axes for readability
binary_matrix = binary_matrix.sort_index().sort_index(axis=1)

# Step 4: Plot heatmap
plt.figure(figsize=(len(binary_matrix.columns)*0.5 + 4, len(binary_matrix)*0.4 + 4))
sb.heatmap(binary_matrix, cmap='Greens', cbar=False, linewidths=0.3, linecolor='gray')

plt.xlabel("AOTU Types")
plt.ylabel("Input ROIs")
plt.title("Input ROIs per Unique AOTU Type")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
